In [1]:
#Importing the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Importing the csv files into dataframes (both train and test)

emnisttrain = pd.read_csv("/content/drive/MyDrive/WOC/emnist-letters-train.csv", header=None)
emnisttest = pd.read_csv("/content/drive/MyDrive/WOC/emnist-letters-test.csv", header=None)

In [3]:
#Functions

def importing(dataframe, examples):
    X = dataframe.iloc[:examples,1:].transpose() #784x4000
    features = X.shape[0]
    examples = X.shape[1]
    Y = dataframe.iloc[:examples,0].to_numpy()
    return X, Y, features, examples

Xtrain_emnist, Ytrain_emnist, trainfeatures_emnist, trainexamples_emnist = importing(emnisttrain, 88800)
Xtest_emnist, Ytest_emnist, testfeatures_emnist, testexamples = importing(emnisttest, 14800)

def scaling(X):
    X = X.div(255)
    X = X.to_numpy()
    return X

Xtrain_emnist = scaling(Xtrain_emnist)
Xtest_emnist = scaling(Xtest_emnist)

def sigmoid(X, theta, bias):
  z = np.dot(theta, X) + bias
  return 1/(1+np.exp(-z))

def onevsall(Y, examples):
  Yzeros = np.zeros((examples, 26))
  for i in range(examples):
    val = Y[i] - 1
    Yzeros[i][val] = 1
  Y = Yzeros
  return Y.T

Ytrain_emnist = onevsall(Ytrain_emnist, trainexamples_emnist)

In [4]:
# def neural(X, Y, theta, examples):
#   z = np.concatenate((np.array([1 for i in range(examples)]).reshape(1,examples),np.dot(theta.T, X))) #17x4000
#   layer = sigmoid(z)

In [5]:
# elementslayer2 = 16
# theta1 = np.random.randn(trainfeatures_emnist, elementslayer2) #784x100
# z = neural(Xtrain_emnist, Ytrain_emnist, theta1)
# print(z.shape)

In [6]:
# class neural:
#   def __init__(self, X, Y, theta, learningrate, iteration, examples, nooflayers, layersize):
#     self.X = X
#     self.Y = Y
#     self.theta = theta
#     self.learningrate = learningrate
#     self.iteration = iteration
#     self.examples = self.X.shape[1]
#     self.nooflayers = nooflayers
#     self.layersize = layersize
#   def onevsall(self):
#     Yzeros = np.zeros((self.examples, 26))
#     for i in range(self.examples):
#         val = self.Y[i] - 1
#         Yzeros[i][val] = 1
#     self.Y = Yzeros
#     return self.Y
#   def sigmoid(self):
#     outp = 1/(1+np.exp(-self.X))
#   def derivativesigmoid(self):
#     return self.sigmoid() * (1 - self.sigmoid())
#   def layers(self, inp):
#     self.inp = inp
#     z = np.concatenate((np.array([1 for i in range(self.examples)]).reshape(1,self.examples),np.dot(self.theta.T, self.inp))) #100x4000
#     layer = self.sigmoid(z)
#     return layer
#   def forward(self):
#     self.theta = np.random.randn(self.X.shape[0], self.layersize[0])
#     layer = self.layers(self.X)
#     for i in range(self.nooflayers):
#       self.theta = np.random.randn(self.layersize[i]-1, self.layersize[i+1])
#       layer.append(self.layers(self.inp))
#     return layer
#   def backward(self):
#     self.Y = self.onevsall()
#     self.ypred = self.forward()
#     der_output = np.dot(self.forward(), (self.Y - self.ypred) * self.derivativesigmoid())
    

In [7]:
# class NeuralNetwork:
#     def __init__(self):
#         self.wij   = np.random.rand(784,100) # input to hidden layer weights
#         self.wjk   = np.random.rand(100,26) # hidden layer to output weights
        
#     def sigmoid(self, x, w):
#         z = np.dot(x, w)
#         return 1/(1 + np.exp(-z))
    
#     def sigmoid_derivative(self, x, w):
#         return self.sigmoid(x, w) * (1 - self.sigmoid(x, w))
    
#     def gradient_descent(self, x, y, iterations):
#         for i in range(iterations):
#             Xi = x
#             Xj = self.sigmoid(Xi, self.wij)
#             yhat = self.sigmoid(Xj, self.wjk)
#             # gradients for hidden to output weights
#             g_wjk = np.dot(Xj.T, (y - yhat) * self.sigmoid_derivative(Xj, self.wjk))
#             # gradients for input to hidden weights
#             g_wij = np.dot(Xi.T, np.dot((y - yhat) * self.sigmoid_derivative(Xj, self.wjk), self.wjk.T) * self.sigmoid_derivative(Xi, self.wij))
#             # update weights
#             self.wij -= 0.5*g_wij
#             self.wjk -= 0.5*g_wjk
#         print('The final prediction from neural network are: ')
#         print(yhat)

# neural_network = NeuralNetwork()
# print('Random starting input to hidden weights: ')
# print(neural_network.wij)
# print('Random starting hidden to output weights: ')
# print(neural_network.wjk)
# X = Xtrain_emnist.T
# y = Ytrain_emnist
# neural_network.gradient_descent(X, y, 1)

In [8]:
class NeuralNetwork:

  def __init__(self):
    self.wij = np.random.randn(100,784)
    self.wjk = np.random.randn(26,100)
    self.b1 = np.zeros((100,1))
    self.b2 = np.zeros((26,1))
    self.examples = Xtrain_emnist.shape[1]

  def sigmoid(self, X, theta, bias):
    z = np.dot(theta, X) + bias
    return 1/(1+np.exp(-z))

  def derivativesigmoid(self, X, theta):
    return self.sigmoid(X, theta) * (1 - self.sigmoid(X, theta))
  
  def model(self, X, Y, lr, iteration):
    costlist = []
    for i in range(iteration):
      Xi = X
      Xj = self.sigmoid(Xi, self.wij, self.b1)
      ypred = self.sigmoid(Xj, self.wjk, self.b2)
      cost = (-1/trainexamples_emnist) * (np.sum(Y*np.log(ypred) + (1-Y)*np.log(1-ypred)))
      costlist.append(cost)
      dz2 = ypred - Y
      dw2 = 1/self.examples * np.dot(dz2, Xj.T)
      db2 = 1/self.examples * np.sum(dz2, axis = 1, keepdims=True)
      dz1 = 1/self.examples * np.dot(self.wjk.T, dz2) * (Xj) * (1 - Xj)
      dw1 = 1/self.examples * np.dot(dz1, Xi.T)
      db1 = 1/self.examples * np.sum(dz1, axis = 1, keepdims=True)
      self.wij -= lr * dw1
      self.wjk -= lr * dw2
      self.b1 -= lr * db1
      self.b2 -= lr * db2
    print(ypred.shape)
    return ypred, self.wij, self.wjk, self.b1, self.b2

In [ ]:
nn = NeuralNetwork()
ypred, thetaij, thetajk, biasij, biasjk = nn.model(Xtrain_emnist, Ytrain_emnist, 1, 500)

In [ ]:
def accuracy():
  Y_pred = np.argmax(ypred, axis=0) + 1
  Y_check = Y_pred - emnisttrain.iloc[:88800,0].to_numpy()
  accuracy = Y_check.tolist().count(0)/88800*100
  return accuracy, Y_pred
accuracy, Y_pred = accuracy()
accuracy

In [ ]:
Y_pred

In [ ]:
def accuracy_final(X, Y):
  Xi = X
  Xj = sigmoid(Xi, thetaij, biasij)
  y_final = sigmoid(Xj, thetajk, biasjk)
  Y_pred = np.argmax(y_final, axis=0) + 1
  Y_check = -(Y_pred - Y)
  accuracy = Y_check.tolist().count(0)/len(Y)*100
  return accuracy, Y_check, Y_pred
accuracy, Y_check, Y_pred = accuracy_final(Xtest_emnist, Ytest_emnist)
accuracy

In [ ]:
# class Neural:

#   def __init__(self, X, Y, hiddennodes):

#     self.X = X
#     self.Y = Y
#     self.inputnodes = self.X.shape[0]
#     self.examples = self.X.shape[1]
#     self.hiddennodes = hiddennodes
#     self.outputnodes = 26

#     self.layers = [self.inputnodes] + self.hiddennodes + [self.outputnodes]
#     self.activations = {0:[]}
#     self.activations[0] = self.X

#     self.derivatives = {}

#     self.weights = {}
#     for i in range(len(self.layers) - 1):
#       theta = np.random.randn(self.layers[i], self.layers[i+1])
#       self.weights[i] = (theta)
  
#     bias = np.ones((1,self.examples))

#   def sigmoid(self, x, w):
#     z = np.dot(w.T, x)
#     return 1/(1+np.exp(-z))
  
#   def pdsigmoid(self, x, w):
#     return self.sigmoid(x, w) * (1 - self.sigmoid(x, w))
  
#   def model(self, lr, iteration):
    
#     for i in range(iteration):

#       for i in range(len(self.layers) - 2):
#         self.activations[i+1] = (self.sigmoid(self.activations[i], self.weights[i]))
      
#       ypred = self.sigmoid(self.activations[len(self.layers) - 2], self.weights[len(self.layers) - 2])
#       self.activations[len(self.layers)-1] = ypred

#       cost = -(1/self.examples) * np.sum(self.Y * np.log(ypred) + (1-self.Y) * (np.log(1 - ypred)))

#       for i in range(len(self.layers) - 2):
        
#         self.weights[len(self.layers)-2-i] -= lr * self.derivatives[len(self.layers)-2-i]

#     return self.weights

In [ ]:
# nn = Neural(Xtrain_emnist, Ytrain_emnist, [100,50])
# nn.model(0.003, 100)